In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!curl -O https://raw.githubusercontent.com/manhtientran/4998-Scripts/master/requirements.txt

!curl -O https://raw.githubusercontent.com/manhtientran/4998-Scripts/master/train.json
!curl -O https://raw.githubusercontent.com/manhtientran/4998-Scripts/master/dev.json
!curl -O https://raw.githubusercontent.com/manhtientran/4998-Scripts/master/test.json

!curl -O https://raw.githubusercontent.com/manhtientran/4998-Scripts/master/Data_for_training_MLM/viquad_mlm_train.txt
!curl -O https://raw.githubusercontent.com/manhtientran/4998-Scripts/master/Data_for_training_MLM/viquad_mlm_dev.txt
!curl -O https://raw.githubusercontent.com/manhtientran/4998-Scripts/master/Data_for_training_MLM/viquad_mlm_test.txt

!curl -O https://raw.githubusercontent.com/manhtientran/4998-Scripts/master/vi_squad-translate-train-train-v1.1.json



In [ ]:
import json
with open("train.json") as f:
    data = json.load(f)

print(data.keys())

In [ ]:
contexts = []
questions = []
answers = []

In [ ]:
contents = data["data"]
print(len(contents))

for i in range(len(contents)):
    paras = contents[i]["paragraphs"]
    for j in range(len(paras)):
        contexts.append(paras[j]["context"])
        quess = paras[j]["qas"]
        for k in range(len(quess)):
            questions.append(quess[k]["question"])
            answers.append(quess[k]["answers"][0]["text"])

In [ ]:
print("# context: ", len(contexts))
print("# question: ", len(questions))
print("# answers: ", len(answers))


In [ ]:
print(answers[0])

In [ ]:
# Check "_" characters
def check_(contents):
  for content in contents:
    if "_" in content:
      print(content)
      print(1)

check_(contexts)
check_(questions)
check_(answers)




# Process for PhoBERT input

In [ ]:
# Install the vncorenlp python wrapper
!pip3 install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [ ]:
# See more details at: https://github.com/vncorenlp/VnCoreNLP

# Load rdrsegmenter from VnCoreNLP
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [ ]:
# EXAMPLE

# Input 
text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

# To perform word (and sentence) segmentation
sentences = rdrsegmenter.tokenize(text) 
print(sentences, "\n")

# for sentence in sentences:
# 	print(" ".join(sentence))
 

preprocessed_text = " ".join(" ".join(sentence) for sentence in rdrsegmenter.tokenize(text))
print(preprocessed_text, "\n")
print(preprocessed_text.split(" "))
print(len(preprocessed_text.split(" ")))


In [ ]:
def segmentate(contents):
    for i in range(len(contents)):
        contents[i] = " ".join(" ".join(sentence) for sentence in rdrsegmenter.tokenize(contents[i]))
    return contents

questions = segmentate(questions)
contexts = segmentate(contexts)
answers = segmentate(answers)


In [ ]:
# max_ans_len = -1
# max_que_len = -1

# for question in questions:
#     max_que_len = max(max_que_len, len(question.split(" ")))

# for answer in answers:
#     max_ans_len = max(max_ans_len, len(answer.split(" ")))

# print("max_que_len :", max_que_len)
# print("max_ans_len :", max_ans_len)


In [ ]:
def printList(arr):
    for i in range(len(arr)):
        print(arr[i], "\n")

# printList(questions)
# print("===========")
# printList(contexts)

In [ ]:
import random

random.shuffle(questions)
random.shuffle(contexts)

# printList(questions)
# print("===========")
# printList(contexts)

In [ ]:
def addElements(arr, interva):
    (start, end) = interva
    res = []
    for i in range(start, end + 1):
        res.append(arr[i])
    return res

In [ ]:
print("# context: ", len(contexts))
print("# question: ", len(questions))

print("total: ", len(contexts) + len(questions))

In [ ]:
# TRAIN-DEV-TEST: 0.8-0.1-0.1

context_intervals = [(0, 3280), (3281, 3691), (3692, 4100)]
question_intervals = [(0, 14863), (14864, 16702), (16703, 18578)]



In [ ]:
viquad_mlm_train_examples = []
viquad_mlm_dev_examples = []
viquad_mlm_test_examples = []

In [ ]:
viquad_mlm_train_examples += addElements(contexts, context_intervals[0])
viquad_mlm_train_examples += addElements(questions, question_intervals[0])

viquad_mlm_dev_examples += addElements(contexts, context_intervals[1])
viquad_mlm_dev_examples += addElements(questions, question_intervals[1])

viquad_mlm_test_examples += addElements(contexts, context_intervals[2])
viquad_mlm_test_examples += addElements(questions, question_intervals[2])

In [ ]:
print("viquad mlm train examples: ", len(viquad_mlm_train_examples))
print("viquad mlm dev examples: ", len(viquad_mlm_dev_examples))
print("viquad mlm test examples: ", len(viquad_mlm_test_examples))

In [ ]:
def getChunks(seq_length, example):
    res = []

    tokens = example.split(" ")
    num_of_tokens = len(tokens)
    if num_of_tokens <= seq_length:
        res.append(example)
    else:
        num_chunk_has_seq_length = num_of_tokens // seq_length
        for i in range(num_chunk_has_seq_length):
            chunk = " ".join(tokens[i * seq_length:i * seq_length + seq_length])
            # print(chunk)

            res.append(chunk)
        if (num_chunk_has_seq_length - 1) * seq_length + seq_length < num_of_tokens:
            chunk = " ".join(tokens[num_chunk_has_seq_length * seq_length: num_of_tokens - 1])
            # print(chunk)

            res.append(chunk)
    return res

# TEST
example = """Ngày 9 tháng 9 năm 1715 , Nhiếp_chính dẫn nhà_vua rời khỏi cung_điện Versailles về Paris , ở đây Nhiếp_chính ở tại tư_gia thuộc Cung_điện hoàng_gia . Ngày 12 tháng 9 , ông ra sắc_lệnh đầu_tiên , khai_mạc Toà_án công_lý lần đầu_tiên dưới triều của mình tại Cung_điện hoàng_gia . Từ tháng 9 năm 1715 đến tháng 1 năm 1716 ông sống tại Chateau de Vincennes , trước khi dời đến Cung_điện Tuileries . Tháng 2 năm 1717 , ông vừa tròn 7 tuổi , và bị tách khỏi người bảo_mẫu là Madame_Ventadour và đặt dưới sự chăm_sóc của Francois de Villeroy , vị Quận công 73 tuổi và Marechal de France , gọi theo chức_vị mà Louis_XIV đã tấn_phong cho ông trong tờ di mệnh tháng 8 năm 1714 . Villeroy hướng_dẫn cho nhà_vua trẻ những lễ_nghi cung_đình , dạy ông cách duyệt_binh , và nghi_thức tiếp khách trong hoàng_tộc . Những vị khách mà ông tiếp bao_gồm Sa hoàng Pyotr_Đại đế năm 1717 ; trái với nghi_thức thông_thường , vị Sa hoàng cao 2 mét đã bế ông lên_tay và hôn ông . Louis cũng học cưỡi ngựa và bắn cung , và nhà_vua trẻ nhanh_chóng say_mê những trò này . Năm 1720 , theo gương Louis_XIV , Villeroy bảo nhà_vua Louis múa trước công_chúng hai điệu ballets tại Cung_điện Tuileries ngày 24 tháng 2 năm 1720 , và một lần nữa trong sự_kiện The_Ballet des Elements ngày 31 tháng 12 năm 1721 . Louis nhút_nhát không thích trò này và ông không bao_giờ múa ballet lần nào nữa . """

getChunks(256, example)

In [ ]:
def transform_to_valid_token_sequence(seq_length, examples):
    res = []
    for i in range(len(examples)):
        tokens = examples[i].split(" ")
        num_of_tokens = len(tokens)
        if num_of_tokens <= seq_length:
            res.append(examples[i])
        else:
            chunks = getChunks(seq_length, examples[i])
            for chunk in chunks:
                res.append(chunk)
    return res

In [ ]:
viquad_mlm_train_examples = transform_to_valid_token_sequence(256, viquad_mlm_train_examples)
viquad_mlm_dev_examples = transform_to_valid_token_sequence(256, viquad_mlm_dev_examples)
viquad_mlm_test_examples = transform_to_valid_token_sequence(256, viquad_mlm_test_examples)

In [ ]:
print(len(viquad_mlm_train_examples))
print(len(viquad_mlm_dev_examples))
print(len(viquad_mlm_test_examples))


# for example in viquad_mlm_train_examples:
#     if len(example) > 0 and example[0].islower():
#         print(example)    


In [ ]:
# !touch /content/drive/MyDrive/IT4998/Data_for_training_MLM/viquad_mlm_train.txt
# !touch /content/drive/MyDrive/IT4998/Data_for_training_MLM/viquad_mlm_dev.txt
# !touch /content/drive/MyDrive/IT4998/Data_for_training_MLM/viquad_mlm_test.txt

In [ ]:
# # WRITE TO FILE
# with open("/content/drive/MyDrive/IT4998/Data_for_training_MLM/viquad_mlm_train.txt", "w") as f:
#     for example in viquad_mlm_train_examples:
#         example = example + "\n"
#         f.write(example)

# with open("/content/drive/MyDrive/IT4998/Data_for_training_MLM/viquad_mlm_dev.txt", "w") as f:
#     for example in viquad_mlm_dev_examples:
#         example = example + "\n"
#         f.write(example)

# with open("/content/drive/MyDrive/IT4998/Data_for_training_MLM/viquad_mlm_test.txt", "w") as f:
#     for example in viquad_mlm_test_examples:
#         example = example + "\n"
#         f.write(example)